In [1]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Attilio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import re

In [3]:
path = r"C:\Users\Attilio\Desktop\progettone\tf_idf\clean_subtitles\Disney"

In [4]:
import os
all_text = []
for r, d, files in os.walk(path):
    for f in files:
        filename = r+'\\\\'+f
        with open(filename, 'r', encoding ='utf-8', errors='ignore') as fh:
            text = fh.read()
            all_text.append(text)

In [5]:
def list_to_string(s):
    string = ""
    for x in s:
        string += x
    return string

In [6]:
all_text = list_to_string(all_text)
all_text = all_text.replace('\n','')
all_text = all_text.replace('\r','')
all_text = all_text.replace('.',' ')

all_text = all_text.lower()
all_text = re.sub(r'[^a-zA-Z]','', all_text)

In [7]:
len(all_text)

9297778

In [8]:
# #saving to a file
# import csv
# with open('all_text.txt', 'w', encoding = 'utf-8', errors = 'ignore') as myfile:
#     writer = csv.writer(myfile)
#     writer.writerows(all_text)

# tokenizzazione

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
stop_words = set(stopwords.words('english'))
table = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}

tokens = word_tokenize(all_text)



In [ ]:
type(stop_words)

In [ ]:
clean_tokens = [w.translate(table).strip() for w in tokens if not w in stop_words]

In [ ]:
len(clean_tokens)

In [ ]:
# #saving to a file
# with open('cleantokens.txt', 'w', encoding = 'utf-8', errors = 'ignore') as myfile:
#     writer = csv.writer(myfile)
#     writer.writerows(clean_tokens)

# sentence splitting

In [ ]:
from nltk import sent_tokenize

In [ ]:
sentences = sent_tokenize(all_text)

In [ ]:
len(sentences)

In [ ]:
sentences[0]

In [ ]:
tok_sents = [word_tokenize(sentence) for sentence in sentences]

In [ ]:
tok_sents[0:3]

In [ ]:
#saving
# with open('tok_sents.txt', 'w', encoding = 'utf-8', errors = 'ignore') as myfile:
#     writer = csv.writer(myfile)
#     writer.writerows(tok_sents)

# Stats

In [ ]:
vocab = set(clean_tokens)
len(vocab)


In [ ]:
from collections import Counter
frequencies = list(Counter(clean_tokens).items())
frequencies[:30]

In [ ]:
frequencies.sort(key=lambda x: x[1], reverse=True)

In [ ]:
for word, frequency in frequencies[:30]:
    print(word,'\t',frequency)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.loglog([i+1 for i,_ in enumerate(frequencies)],[f for w,f in frequencies])
plt.show()

# WORD2VEC

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(tok_sents, size=100, window=10, min_count=15, sg=0, iter=20, negative=10)

In [ ]:
#learned vocabulary
words = list(w2v_model.wv.vocab)
len(words)

In [ ]:
w2v_model.wv.most_similar(['aurora'])

In [ ]:
w2v_model.wv.most_similar(['king', 'boy'], ['majesty'])

In [ ]:
w2v_model.wv.similarity('king', 'prince')

# constructin matrix for graph

In [ ]:
#modello w2v
voc = w2v_model.wv.vocab


In [ ]:
len(voc)

In [ ]:
#indicizzazione delle keys del modello
indici = dict()
contatore = 0
for key in voc:
    if key not in indici:
        indici[key] = contatore
        contatore += 1

In [ ]:
indici = dict()

for index, key in enumerate(voc):
    if key not in indici:
        indici[key] = index


In [ ]:
reverse_indici = dict()
for term, termID in indici.items():
    reverse_indici[termID] = term

In [ ]:
#costruzione matrice
graph_matrix = np.zeros(shape=(len(voc), len(voc)), dtype=np.float32)

In [ ]:
for index1, key1 in enumerate(voc):
    for index2, key2 in enumerate(voc):
        graph_matrix[index1 ][index2] = w2v_model.wv.similarity(key1, key2)

In [ ]:
#creazione lista di tuple

In [ ]:
graph_matrix[7000][7028]

In [ ]:
type(graph_matrix)

In [ ]:
#saving the matrix (binary data)
np.save('graph_matrix.npy', graph_matrix)

In [ ]:
#Human readable data
np.savetxt('graph_matrix.txt', graph_matrix)

In [ ]:
#loading the matrix
a = np.load('graph_matrix.npy')

In [ ]:
len(graph_matrix)

In [ ]:
#constructing the list of tuples
graph_list = []
for x in range(len(graph_matrix)):
    for y in range(len(graph_matrix)):
        if graph_matrix[x][y] >= 0.4:
            graph_list.append((reverse_indici[x],reverse_indici[y],graph_matrix[x][y]))       
        else:
            continue

In [ ]:
graph_list

In [ ]:
import csv
with open('graph_disney.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Source','Target','Weight'])
    writer.writerow(graphlist)

# PLOTTING

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, n_iter=1000)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(w2v_model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')






In [ ]:
import pandas as pd
d = {'x': [x_vals], 'y': [y_vals], 'token':[labels]}
df = pd.DataFrame(data=d)


In [ ]:
subset_df = df.sample(n=200, replace=True)

In [ ]:
x = list(subset_df['x'])
y = list(subset_df['y'])
token = list(subset_df['token'])

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar, ColumnDataSource
from bokeh.io import export_svgs

In [ ]:
mapper = linear_cmap(field_name='y', palette=Spectral6 ,low=min(y_vals) ,high=max(y_vals))

source = ColumnDataSource(dict(x=x,y=y))

In [ ]:
p = figure(plot_width=800, plot_height=800)
_ = p.text(x_vals, y_vals, labels, text_color=mapper,color=mapper)
show(p)


In [ ]:
# WORD ALGEBRA
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = w2v_model.wv.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [ ]:
word_algebra(add=['king'], subtract=['majesty'])

In [ ]:
word_algebra(add=['prince', 'majesty'])

In [ ]:
word_algebra(['daughter', 'princess'])

## FastText

In [ ]:
from gensim.models import FastText
fast_model = FastText(tok_sents,size=100, window=10, min_count=5, sg=1, iter=20, negative=10)

In [ ]:
fast_model.wv.most_similar(['apple'])

In [ ]:
fast_model.wv.most_similar(['king'])

In [ ]:
fast_model.wv.most_similar(['mirror'])

In [ ]:
fasttext_wAp_analogy = fast_model.wv.evaluate_word_analogies(test_file, dummy4unknown=True)

# google precomputed word embeddings

In [ ]:
import gensim
google_w2v_file = 'GoogleNews-vectors-negative300.bin'
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_w2v_file,binary=True)

In [ ]:
google_model.most_similar(['king','female'],['male'])

In [ ]:
w2v_large_analogy = google_model.wv.accuracy(test_file)